In [70]:
import requests
import json
import time

In [23]:
# https://github.com/philipperemy/name-dataset
# !pip3 install names-dataset

In [24]:
from names_dataset import NameDataset, NameWrapper

In [69]:
api_key = "16831ee8bemshcddf2990350ca81p1d6a5fjsnbc1bd82e3091"

In [26]:
nd = NameDataset() 

In [27]:
name_value = 3
sleep_time = 0.5 

In [28]:
# user data download 

In [29]:
# load country data 
try:
    with open('regions.json', 'r') as voivodeships:
        voivodeships_data = json.load(voivodeships)
except Exception as e:
    print(e)

In [30]:
voivodeships_data

{'PL': [{'name': 'Mazowsze', 'cities': ['warsaw', 'lodz']},
  {'name': 'Wielkopolska', 'cities': ['poznan', 'szczecin']},
  {'name': 'Małopolska', 'cities': ['krakow', 'rzeszow']},
  {'name': 'Śląsk', 'cities': ['katowice', 'gliwice']},
  {'name': 'Pomorze', 'cities': ['gdansk', 'gdynia']}],
 'GB': [{'name': 'England', 'cities': ['london', 'birmingham']},
  {'name': 'Scotland', 'cities': ['glasgow', 'edinburgh']},
  {'name': 'Wales', 'cities': ['cardiff', 'swansea']},
  {'name': 'Northern Ireland', 'cities': ['belfast', 'londonderry']}]}

In [31]:
users = {}

In [32]:
for country_key in list(voivodeships_data.keys()):
    users.update({country_key : {} })
    for voivodeship in voivodeships_data[country_key]:
        users[country_key].update({voivodeship["name"] : {}})
        for city in voivodeship["cities"]:
            users[country_key][voivodeship["name"]].update({city :[]})   

In [33]:
users

{'PL': {'Mazowsze': {'warsaw': [], 'lodz': []},
  'Wielkopolska': {'poznan': [], 'szczecin': []},
  'Małopolska': {'krakow': [], 'rzeszow': []},
  'Śląsk': {'katowice': [], 'gliwice': []},
  'Pomorze': {'gdansk': [], 'gdynia': []}},
 'GB': {'England': {'london': [], 'birmingham': []},
  'Scotland': {'glasgow': [], 'edinburgh': []},
  'Wales': {'cardiff': [], 'swansea': []},
  'Northern Ireland': {'belfast': [], 'londonderry': []}}}

In [72]:
def get_popular_names(country, name_value=name_value, nd=nd):
    top_names = nd.get_top_names(n=name_value, gender='Male', country_alpha2=country)
    top_names[country]["F"] = nd.get_top_names(n=name_value, gender='Female', country_alpha2=country)[country]["F"]
    return top_names

In [73]:
def get_users(city, name, offset=None, api_key=api_key):
    url = "https://soundcloud-scraper.p.rapidapi.com/v1/search/users"

    if not offset:
        querystring = {"term":name,"place":city}
    else:
        querystring = {"term":name,"offset":offset,"place":city}
    
    headers = {
    	"X-RapidAPI-Key": api_key,
    	"X-RapidAPI-Host": "soundcloud-scraper.p.rapidapi.com"
    }
    
    response = requests.get(url, headers=headers, params=querystring)
    
    return response.json()

In [74]:
def get_users_for_country(users, country, top_names, offset=None, api_key=api_key): 
    names = top_names[country]['M'] + top_names[country]['F']
    for voivodeship in users[country].keys():
        for city in users[country][voivodeship].keys():
            for name in names:
                print(f"looking for users with name {name} in {city}")
                users_result = get_users(city, name)
                if 'totalCount' in users_result:
                    cnt = users_result['totalCount']
                    print(f"got {cnt if cnt <= 50 else 50} of maximum {cnt} possible" )
                    users[country][voivodeship][city] += users_result["users"]["items"]
                # save_to_file(f"users_{name}_{city}_{country}", users)    
                    time.sleep(sleep_time)
        
    return users

In [75]:
def get_user_likes(userlinkname, offset=None, api_key=api_key):
    url = "https://soundcloud-scraper.p.rapidapi.com/v1/user/likes"    
    querystring = {"user":f"https://soundcloud.com/{userlinkname}"}    
    headers = {
    	"X-RapidAPI-Key": api_key,
    	"X-RapidAPI-Host": "soundcloud-scraper.p.rapidapi.com"
    }    
    response = requests.get(url, headers=headers, params=querystring)

    return response.json()


In [76]:
def get_users_likes(users, likes):
    i = 1
    for country in users.keys():
        for voivideship in users[country].keys():
            for city in users[country][voivideship].keys():
                for user in users[country][voivideship][city]:
                    userlinkname = user["permalink"].split('/')[-1]
                    print(f"user {userlinkname}")
                    user_likes = get_user_likes(userlinkname)
                    print(f"likes of user no: {i} , {len(user_likes)}")
                    i+=1
                    time.sleep(sleep_time)
                    if user_likes != None:  
                        if 'likes' in user_likes:
                            likes[userlinkname] = user_likes["likes"]["items"] 
                            if i % 10 == 0:    
                                save_to_file(f"likes_{i}_{userlinkname}_{city}_{country}", likes)
                    
    return likes        

In [77]:
def save_to_file(filename, file_to_save):
    with open(f"{filename}.json", "w") as outfile: 
        json.dump(file_to_save, outfile, indent = 4)

In [39]:
users.keys()

dict_keys(['PL', 'GB'])

In [40]:
users

{'PL': {'Mazowsze': {'warsaw': [], 'lodz': []},
  'Wielkopolska': {'poznan': [], 'szczecin': []},
  'Małopolska': {'krakow': [], 'rzeszow': []},
  'Śląsk': {'katowice': [], 'gliwice': []},
  'Pomorze': {'gdansk': [], 'gdynia': []}},
 'GB': {'England': {'london': [], 'birmingham': []},
  'Scotland': {'glasgow': [], 'edinburgh': []},
  'Wales': {'cardiff': [], 'swansea': []},
  'Northern Ireland': {'belfast': [], 'londonderry': []}}}

In [41]:
users = get_users_for_country(users, 'PL', get_popular_names("PL"))

looking for users with name Piotr in warsaw
got 50 of maximum 412 possible
looking for users with name Marcin in warsaw
got 50 of maximum 392 possible
looking for users with name Krzysztof in warsaw
got 50 of maximum 162 possible
looking for users with name Anna in warsaw
got 50 of maximum 185 possible
looking for users with name Agnieszka in warsaw
got 50 of maximum 87 possible
looking for users with name Katarzyna in warsaw
got 50 of maximum 97 possible
looking for users with name Piotr in lodz
got 50 of maximum 120 possible
looking for users with name Marcin in lodz
got 50 of maximum 95 possible
looking for users with name Krzysztof in lodz
got 50 of maximum 52 possible
looking for users with name Anna in lodz
got 49 of maximum 49 possible
looking for users with name Agnieszka in lodz
got 25 of maximum 25 possible
looking for users with name Katarzyna in lodz
got 22 of maximum 22 possible
looking for users with name Piotr in poznan
got 50 of maximum 176 possible
looking for users wi

In [42]:
users

{'PL': {'Mazowsze': {'warsaw': [{'type': 'user',
     'id': 133933703,
     'permalink': 'https://soundcloud.com/piotr-ho',
     'createdAt': '2015-01-19T20:39:26Z',
     'lastModified': '2023-11-16T19:28:13Z',
     'name': 'Piotr Ho',
     'description': 'contact: piotr.horelik@gmail.com\n\nPiotr Ho is a Warsaw DJ and promoter active on the scene since 2016. Organization of events integrating various genres, from downtempo, through house and techno, to psychedelic trance music, confirmed his need for a cross-sectional view on music. It resulted in regular performances at various events, where he presents his favorite sounds selection.\nThe mystical atmosphere of the community, full of symbiosis between the artist and the dance floor, as well as the infinity of new "forms" in electronic music, is the basis for the musical existence of the Warsaw artist. Superiority of a deep groove over the pace with a rough kick, is a principle he follows to keep the hypnotic atmosphere. His favourite

In [45]:
# save_to_file("saved_users", users)

In [46]:
users = get_users_for_country(users, 'GB', get_popular_names("GB"))

looking for users with name David in london
got 50 of maximum 4210 possible
looking for users with name Paul in london
got 50 of maximum 3958 possible
looking for users with name John in london
got 50 of maximum 4039 possible
looking for users with name Sarah in london
got 50 of maximum 1620 possible
looking for users with name Emma in london
got 50 of maximum 1169 possible
looking for users with name Laura in london
got 50 of maximum 1126 possible
looking for users with name David in birmingham
got 50 of maximum 407 possible
looking for users with name Paul in birmingham
got 50 of maximum 404 possible
looking for users with name John in birmingham
got 50 of maximum 447 possible
looking for users with name Sarah in birmingham
got 50 of maximum 155 possible
looking for users with name Emma in birmingham
got 50 of maximum 123 possible
looking for users with name Laura in birmingham
got 50 of maximum 135 possible
looking for users with name David in glasgow
got 50 of maximum 943 possible


In [47]:
save_to_file("saved_users", users)

In [79]:
likes = {}

In [67]:
# get_user_likes('david-morris-340740716')
# user_likes = get_user_likes("ramzes-958123791")

In [81]:
likes = get_users_likes(users, likes)

user piotr-ho
likes of user no: 1 , 4
user soundteller-records
likes of user no: 2 , 4
user piotr-dylewski
likes of user no: 3 , 4
user piotr-wiese
likes of user no: 4 , 4
user schmeltzerr
likes of user no: 5 , 4
user saintpeterdjpl
likes of user no: 6 , 4
user piotrexx
likes of user no: 7 , 4
user piotr-balasiewicz
likes of user no: 8 , 4
user piotr-ruci-ski
likes of user no: 9 , 4
user piotr-stechbi-stechbart
likes of user no: 10 , 4
user user-851406428
likes of user no: 11 , 4
user piotrpl
likes of user no: 12 , 4
user qbs
likes of user no: 13 , 4
user piotrgajcymusic
likes of user no: 14 , 4
user djpd-1
likes of user no: 15 , 4
user janusz-prusinowski-kompania
likes of user no: 16 , 4
user piotrrosciszewski
likes of user no: 17 , 4
user hornad
likes of user no: 18 , 4
user subrosedub
likes of user no: 19 , 4
user dziun
likes of user no: 20 , 4
user deersky
likes of user no: 21 , 4
user petedeepmusic
likes of user no: 22 , 4
user nmls
likes of user no: 23 , 4
user that-simple
likes 

KeyboardInterrupt: 

In [ ]:
save_to_file("saved_likes", likes)

In [82]:
users2 = users.copy()

In [83]:
del users2['PL']

In [ ]:
likes = get_users_likes(users2, likes)

user david-morris-340740716
likes of user no: 1 , 4
user davidbowieofficial
likes of user no: 2 , 4
user thehouseofdisco
likes of user no: 3 , 4
user joy-orbison
likes of user no: 4 , 4
user markknight
likes of user no: 5 , 4
user david-dobell-372493004
likes of user no: 6 , 4
user snatchrecords
likes of user no: 7 , 4
user theofficialdesc
likes of user no: 8 , 4
user jacobcollier
likes of user no: 9 , 4
user harrison
likes of user no: 10 , 4
user sexshooters
likes of user no: 11 , 4
user yesofficial
likes of user no: 12 , 4
user diviad_1
likes of user no: 13 , 4
user davidclarkallen
likes of user no: 14 , 4
user oatmilkandcodeine
likes of user no: 15 , 4
user david-hoade-150655607
likes of user no: 16 , 4
user dj-callmedave
likes of user no: 17 , 4
user davidzowie
likes of user no: 18 , 4
user djreevesuk8701
likes of user no: 19 , 4
user user-688789350
likes of user no: 20 , 4
user david-toop
likes of user no: 21 , 4
user david_taro
likes of user no: 22 , 4
user talayrileynow
likes of